# Statistical Modeling in Python

### Customization vs Rapid Development

As we know from (painful?) experience, Python is powerful because of its ability to leverage `numpy` and `scipy` to implement any statistical model from scratch. We can write the requisite matrix algebra, or the relevant likelihood function, and from there can optimize our model, calculate confidence intervals, and report the output of that model through data frames, lists, or printed tables. Building our own models is great! We get to build a model based on the exact context and assumptions of our problem, and therefore get exactly the model that we wanted. Unfortunately, it takes a LOT of time!

This lesson will provide our first exposure to pre-written statistical modeling in Python. We will be able to use only a couple of lines of code to implement complex and valuable statistical and machine learning models. Because the most costly asset in programming is the time that we spend debugging and writing code (running code is MUCH faster and cheaper than the time spent writing code), we are always looking for ways to avoid writing code that someone else has already written.

`statsmodels` is a library that covers the majority of regression models commonly used by economists and statisticians in other fields.

`sklearn` is an analogous library that covers machine learning models (aside from deep neural networks, which have their own implementations).

Each of these libraries is highly optimized to provide performant implementations of models that we use regularly, and allow us to avoid writing these models from scratch unless we need to customize our model for some specific use case! This is great news! You'll never have to think about writing your own linear or logistic regression from scratch again!

Let's dive in.


## Statsmodels

`statsmodels` makes statistics in Python easy! The library contains tools for regressions ranging from linear regression, to logistic regression, count regressions (negative binomial and poisson), various options for robust covariance measures, and tools to implement time series models as well! There are also really useful tools for assisting in creating our regression model based on any structure that best suits us.

We can import `statsmodels` in one of two ways:

1) With support for R-style formulas:

In [ ]:
import statsmodels.formula.api as sm

    /opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
      import pandas.util.testing as tm


This is probably the best way to import our data if we are doing regression analysis for causal inference. In these cases, we are not typically trying to make predictions as new data arrives, and so we do not need to have tools ready to analyze new data using our existing regression models.

2) Import `statsmodels` to use pre-built numpy arrays as inputs:

In [ ]:
import statsmodels.api as sm

In this case, we have other tools that we can use, but we need to manually arrange our `x` and `y` matrices. It looks clunky at first, but can be useful when we are building predictive pipelines using regression models, or when we might want to use both `statsmodels` and `sklearn` with the same data source.

Let's start with option 1...

### Preparing a Dataset

When using formulas, we prepare our dataset by importing the data into a Pandas `DataFrame`. We should take care that each of our variables has a name with
1) **No spaces**
2) No symbols
3) Made up of letters and numbers (also can't have a number as the first character)

Our code so far might look something like:

In [ ]:
import statsmodels.formula.api as smf
import pandas as pd, numpy as np

data = pd.read_csv("https://github.com/dustywhite7/Econ8320/blob/master/AssignmentData/assignment8Data.csv?raw=true")

Assuming that our data set has already been cleaned. If our data has not yet been cleaned, then we need to clean our data prior to working with either `statsmodels` or `sklearn`. This is because regression AND machine learning models require that all information be provided in numeric format. We need to transform text-based data into categorical data (using either ordered numeric columns or binary variable columns generated from our categories), and ensure that all data is represented in the way that we want to use it within our model.

### Regression Equations

`statsmodels` incorporates `R`-style regression equations by using the `patsy` library behind the scenes. We will talk more about `patsy` soon. The pattern for regression equations is as follows:

```"dependent variable ~ independent variable + another independent variable + any other independent variables"```

The regression equation will be stored in a string (unlike in `R`), and we put our dependent variable (also called the endogenous variable, or outcome of interest) in the leftmost position within the string. We separate the dependent variable from all independent (exogenous or explanatory) variables using the `~` symbol. Then, each independent variable is separated from the others using `+` operators.

The reason is is so important that our column names be properly cleaned before implementing regression analysis is that spaces and other problematic formats for column names will cause problems with our regression equations.

### Implementing a Model

The first model we might try is a simple linear regression. These are the most common regression models, and typically what someone is referring to when they discuss "running a regression". The code is wonderfully simple:

In [ ]:
reg = smf.ols("hhincome ~ year", data=data).fit()
print(reg.summary())

                                OLS Regression Results                            
    ==============================================================================
    Dep. Variable:               hhincome   R-squared:                      -0.000
    Model:                            OLS   Adj. R-squared:                 -0.000
    Method:                 Least Squares   F-statistic:                      -inf
    Date:                Wed, 16 Mar 2022   Prob (F-statistic):                nan
    Time:                        15:24:28   Log-Likelihood:            -1.7131e+05
    No. Observations:               13712   AIC:                         3.426e+05
    Df Residuals:                   13711   BIC:                         3.426e+05
    Df Model:                           0                                         
    Covariance Type:            nonrobust                                         
    ==============================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
    ------------------------------------------------------------------------------
    Intercept      0.0188      0.000    138.414      0.000       0.019       0.019
    year          37.8564      0.274    138.414      0.000      37.320      38.392
    ==============================================================================
    Omnibus:                     9819.620   Durbin-Watson:                   1.027
    Prob(Omnibus):                  0.000   Jarque-Bera (JB):           250725.793
    Skew:                           3.151   Prob(JB):                         0.00
    Kurtosis:                      22.978   Cond. No.                     9.31e+17
    ==============================================================================
    
    Warnings:
    [1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
    [2] The smallest eigenvalue is 6.41e-26. This might indicate that there are
    strong multicollinearity problems or that the design matrix is singular.


    /opt/conda/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1657: RuntimeWarning: divide by zero encountered in double_scalars
      return self.ess/self.df_model


When we run these two lines of code, we are creating, fitting, and reporting on a regression model! It's fast, it's clean, and it's really easy to implement! `sm.ols` is the OLS class of regression models, and takes two required arguments: a regression equation (passed as a string), and a data source (expected to be a `pandas.DataFrame` object). We use the `.fit()` method to complete all of the math that actually solves our regression model. When we call `.summary()` on a fitted regression, we get a printout of the regression summary tables for the model, complete with diagnostic measures, estimates of our beta coefficients, and confidence intervals!

If the model is satisfactory, then we are done! (It really is that simple!)

If I want to keep iterating on my model, I might want to try regressing year on the logged average household incomes:

In [ ]:
reg = smf.ols("np.log(hhincome) ~ year", data=data[data['hhincome']>0]).fit()
print(reg.summary())

                                OLS Regression Results                            
    ==============================================================================
    Dep. Variable:       np.log(hhincome)   R-squared:                      -0.000
    Model:                            OLS   Adj. R-squared:                 -0.000
    Method:                 Least Squares   F-statistic:                      -inf
    Date:                Wed, 16 Mar 2022   Prob (F-statistic):                nan
    Time:                        15:31:18   Log-Likelihood:                -17363.
    No. Observations:               13653   AIC:                         3.473e+04
    Df Residuals:                   13652   BIC:                         3.474e+04
    Df Model:                           0                                         
    Covariance Type:            nonrobust                                         
    ==============================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
    ------------------------------------------------------------------------------
    Intercept   2.698e-06   1.82e-09   1481.190      0.000    2.69e-06     2.7e-06
    year           0.0054   3.67e-06   1481.190      0.000       0.005       0.005
    ==============================================================================
    Omnibus:                     5172.537   Durbin-Watson:                   1.277
    Prob(Omnibus):                  0.000   Jarque-Bera (JB):            63678.616
    Skew:                          -1.469   Prob(JB):                         0.00
    Kurtosis:                      13.164   Cond. No.                     8.12e+17
    ==============================================================================
    
    Warnings:
    [1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
    [2] The smallest eigenvalue is 8.4e-26. This might indicate that there are
    strong multicollinearity problems or that the design matrix is singular.


    /opt/conda/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1657: RuntimeWarning: divide by zero encountered in double_scalars
      return self.ess/self.df_model


As you can see from the code above, everything is the same, except that we were able to transform household income using `np.log` on the go! We don't even need to create a new column! We can just do it inside of our regression model! We also subset our data so that the log operator doesn't break our model by introducing $-\infty$ as a possible `hhincome` value.

In other cases, it might be useful to create state-level fixed effects by including dummy variables for the states in our `statefip` column. Note that this won't work with our current data, since we only have one state in our data set.

In [ ]:
reg = smf.ols("np.log(hhincome) ~ year + C(statefip)", data=data[data['hhincome']>0]).fit()
print(reg.summary())

                                OLS Regression Results                            
    ==============================================================================
    Dep. Variable:       np.log(hhincome)   R-squared:                      -0.000
    Model:                            OLS   Adj. R-squared:                 -0.000
    Method:                 Least Squares   F-statistic:                      -inf
    Date:                Wed, 16 Mar 2022   Prob (F-statistic):                nan
    Time:                        15:32:19   Log-Likelihood:                -17363.
    No. Observations:               13653   AIC:                         3.473e+04
    Df Residuals:                   13652   BIC:                         3.474e+04
    Df Model:                           0                                         
    Covariance Type:            nonrobust                                         
    ==============================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
    ------------------------------------------------------------------------------
    Intercept   2.698e-06   1.82e-09   1481.190      0.000    2.69e-06     2.7e-06
    year           0.0054   3.67e-06   1481.190      0.000       0.005       0.005
    ==============================================================================
    Omnibus:                     5172.537   Durbin-Watson:                   1.277
    Prob(Omnibus):                  0.000   Jarque-Bera (JB):            63678.616
    Skew:                          -1.469   Prob(JB):                         0.00
    Kurtosis:                      13.164   Cond. No.                     8.12e+17
    ==============================================================================
    
    Warnings:
    [1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
    [2] The smallest eigenvalue is 8.4e-26. This might indicate that there are
    strong multicollinearity problems or that the design matrix is singular.


    /opt/conda/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1657: RuntimeWarning: divide by zero encountered in double_scalars
      return self.ess/self.df_model


The `C()` command indicates that we would like to consider the `statefip` variable as a **C**ategorical variable, not a numeric variable. We can transform ANY column using the categorical operator. It is most useful when a column is text-based, or when a column is numeric but should not be treated as a count, ordinal, or continuous variable. We CAN use it on our dependent variable, but this will (unless our dependent variable was binary text data) break our regression model, which expects only a single dependent variable, rather than an array of dependent variables.

Sometimes we want to include transformed variables in our model without creating a new column. The `I()` operator allows us to do just that:

In [ ]:
# Square a variable using the I() function for
#   mathematical transformations
reg = smf.ols("np.log(hhincome) ~ age + I(age**2)", data=data).fit()

In this case, we transform `age` by squaring it (maybe in preparation to create an age-earnings profile?). One line, simple syntax, what could be better?

In [ ]:
# Combine variables using the I() function for
#   mathematical transformations
reg = smf.ols("np.log(hhincome) ~ I(age-education-5)", data=data).fit()

This example combines TWO columns to create a new measure (proxying experience by subtracting education from age, and subtracting an additional 5 years). All we have to do is describe the relationship that we want to model as an explanatory variable, and we are off to the races! Most operators are fair game, and we can include an arbitrary number of columns in our measure calculation.

### More robust modeling

If we want to utilize robust standard errors, we can easily update our regression results:

In [ ]:
reg = smf.ols("np.log(hhincome) ~ year + C(statefip)", data=data).fit()
# Use White's (1980) Standard Error
reg.get_robustcov_results(cov_type='HC0')
print(reg.summary())

Or, if we want to cluster our standard errors by state,

In [ ]:
reg = smf.ols("np.log(hhincome) ~ year + C(statefip)", data=data).fit()
# Use Cluster-robust Standard Errors
reg.get_robustcov_results(cov_type='cluster', groups=data['statefip']) # Need to specify groups
print(reg.summary())

We don't have to stick to just `HC0` and cluster-robust standard errors. Below are some of the [covariance options](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.get_robustcov_results.html) that we have:
1) `HC0`: White's (1980) Heteroskedasticity robust standard errors
2) `HC1`, `HC2`, `HC3`: MacKinnon and White's (1985) alternative robust standard errors, with `HC3` being designed for improved performance in small samples
3) `cluster`: Cluster robust standard errors
4) `hac-panel`: Panel robust standard errors

We should choose the standard errors that best fit our specific data needs, and it is important to realize that this choice is highly context-dependent. The structure and nature of our data should be carefully considered, as should the specific regression model that we are trying to implement.

### Time Series Models

Not only can we model linear regression, we also have multiple time series options available. We won't go into much detail, since each of these models deserve to have significant time devoted to them, and we just don't have the time in this class.

- [ARIMA](http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMA.html) models
- [VAR](http://www.statsmodels.org/dev/generated/statsmodels.tsa.vector_ar.var_model.VAR.html) models
- [Exponential Smoothing](https://www.statsmodels.org/stable/tsa.html#exponential-smoothing) models

We can run an ARIMA, for example, using code like the following example:

In [ ]:
# This won't work unless we have multiple years of data (which we currently don't)

from statsmodels.tsa.arima_model import ARIMA

y = data.loc[data['statefip']==31, ['hhincome','year']]
y.index=pd.to_datetime(y.year)
reg = ARIMA(y['hhincome'], order=(1,1,0)).fit()
print(reg.summary())

### Modeling Discrete Outcomes

If we have a [binary dependent variable](https://www.statsmodels.org/devel/discretemod.html), we are able to use either [Logit](https://www.statsmodels.org/devel/generated/statsmodels.discrete.discrete_model.Logit.html#statsmodels.discrete.discrete_model.Logit) or [Probit](https://www.statsmodels.org/devel/generated/statsmodels.discrete.discrete_model.Probit.html#statsmodels.discrete.discrete_model.Probit) models to estimate the effect of exogenous variables on our outcome of interest. To fit a Logit model:

In [ ]:
import statsmodels.api as sm

myformula="married ~ hhincome + C(statefip) + C(year) + educ"
model= sm.Logit.from_formula(myformula, data=data).fit()

### Modeling Count Data

When modeling count data, we have options such as [Poisson](http://www.statsmodels.org/dev/generated/statsmodels.discrete.discrete_model.Poisson.html#statsmodels.discrete.discrete_model.Poisson) and [Negative Binomial](http://www.statsmodels.org/dev/generated/statsmodels.discrete.discrete_model.NegativeBinomial.html#statsmodels.discrete.discrete_model.NegativeBinomial) models.

In [ ]:
data = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/master/DataSets/auto-mpg.csv")

myformula="nchild ~ hhincome + C(statefip) + C(year) + educ + married"

model= sm.Poisson.from_formula(myformula, data=data).fit()

There are many other regression "flavors", and the best way to learn about what is available through `statsmodels` is to [read the docs](https://www.statsmodels.org/stable/user-guide.html).

## The `patsy` library

We have been using regression equations in `statsmodels` a lot without really discussing what is happening behind the scenes. `statsmodels` relies on a library called `patsy` to parse regression equations and prepare our data for regression analysis. While `statsmodels` does a great job of incorporating the `patsy` library for us, this isn't always the case. In fact, it is a really valuable tool in many other contexts (think machine learning or deep learning).


### Why use `patsy`?

We don't necessarily have to use `patsy`. We could just select our variables manually. Creating a column of ones to serve as our intercept column is trivial (you of course remember that from the linear regression assignment). `patsy` is a tool for creating a standardized pipeline to deal with data that is stored in identical formats, and aids us in creating reusable or replicable code. Patsy allows us to separate our endogenous and exogenous variables AND to
	- "Dummy out" categorical variables
	- Easily transform variables (square, or log transforms, etc.)
	- Use identical transformations on future data
    
Even better, `patsy` is just as easy to use as regression equations. We just need to learn about the function wrappers that are necessary to create our processed data:

In [ ]:
import patsy as pt
import pandas as pd
import numpy as np

data = pd.read_csv("https://github.com/dustywhite7/Econ8320/blob/master/AssignmentData/assignment8Data.csv?raw=true")

# To create y AND x matrices
y, x = pt.dmatrices("hhincome ~ year + educ + married + age", data = data)

In order to get started, we need to import `patsy`, and we typically give it the two-letter abbreviation `pt`. Once we have imported our data, we use the `pt.dmatrices` function. This function takes a regression equation (again, as a string), and a data source. The returned value is a **tuple** of `y` and `x`. We can break that tuple into two values by using the `y, x = ...` syntax, so that we have a `y` array and an `x` array.

We don't have to create BOTH `y` and `x` data, though! We can use the `pt.dmatrix` function to just create an `x` matrix. Maybe we already have a dependent variable, and want to try out variations on our explanatory variables to see how each performs. In this case, our regression equation should have no column name to the left of the `~` symbol:

In [ ]:
# To create ONLY an x matrix
x = pt.dmatrix("~ year + educ + married + age", data = data)

One more note is that these regression equations automatically include an intercept term. If you do NOT want an intercept term (some regression models and most machine learning models don't use them), then you can add `-1` as an exogenous variable in your regression equation, in order to indicate that you want to eliminate the column of ones that make up the intercept column in our matrix of exogenous regressors.

### Categorical Variables

Again, we have the functions described in the regression section above available to us as we transform our data. We can create categorical variables:

In [ ]:
# To create y AND x matrices
eqn = "hhincome ~ C(year) + educ + married + age"
y, x = pt.dmatrices(eqn, data = data)

And (again) we can transform variables!

In [ ]:
# To create y AND x matrices
eqn = "I(np.log(hhincome)) ~ C(year) + educ + married + age + I(age**2)"
y, x = pt.dmatrices(eqn, data = data)

We can also use interaction operators. `*` will interact each value of two columns, and also include the original columns in the regression model. `:` will include only the interaction terms, while omitting the original columns. Check out the [explanation of formulas](https://patsy.readthedocs.io/en/latest/formulas.html) for more details.


### SUPER IMPORTANT $\rightarrow$ Same Transformation on New Data!

Often, we will want to build a model with observed data that can make predictions about new observations as those observations are recorded. `patsy` provides a simple function to take the structure of one exogenous matrix and generate another identically structured matrix using new data:

In [ ]:
# To create a new x matrix based on our previous version
xNew = pt.build_design_matrices([x.design_info], dataNew)

In other words, we can create a new matrix in the SAME SHAPE as our original `x` matrix by using the `build_design_matrices()` function in `patsy`.

We pass a list containing the old design matrix information (because we can actually create many matrices simultaneously), as well as the new data from which to construct our new matrix.

Why does recreating our `x` array matter? This process ensures that we always have the same number of categories in our categorical variables. A new, smaller subset of data that is freshly observed may not contain observations of every category, in which case an updated patsy matrix would not contain the correct number of columns! We are able to maintain consistency in our model, making our work replicable. Most importantly, this will streamline the use of `statsmodels` and `sklearn` in the same workflow!

Speaking of `sklearn`...

## `sklearn`

What `statsmodels` does for regression analysis, `sklearn` does for predictive analytics and machine learning. It is a truly fabulous library. `sklearn` is likely the most popular machine learning library, and has a standard API to make using the library VERY simple. Even better, it's documentation is some of the nicest documentation you will find anywhere, and contains incredible detail about how to implement models, as well as lessons about the "how" and "why" of using each model. You couldn't write a better textbook about machine learning than the documentation for `sklearn`.

Below, we will briefly discuss some of the models that are most commonly utilized from `sklearn`. Details will be sparse. We are mostly focused on the code implementation of these models. More detail on how machine learning models work is provided in  Business Forecasting, and is outside the scope of this course.

### Decision Tree Classification (and Regression)

[Classification](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier) and [Regression](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor) Trees (CARTs) are the standard jumping-off point for exploring machine learning. They are very easy to implement in `sklearn`:

In [ ]:
from sklearn import tree
from sklearn.metrics import accuracy_score
import pandas as pd
import patsy as pt

data = pd.read_csv("https://github.com/dustywhite7/pythonMikkeli/raw/master/exampleData/roomOccupancy.csv")

y, x = pt.dmatrices("Occupancy ~ CO2", data=data)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(x, y.squeeze())

pred = clf.predict(x)

print("In-sample accuracy: {}".format(accuracy_score(y.squeeze(), pred)))

    In-sample accuracy: 0.9753162225224119


### Support Vector Machines

We also implement [Support Vector Machines](http://scikit-learn.org/stable/modules/svm.html#svm) for both [classification](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) and [regression](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR):

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

clf = svm.SVC()
clf = clf.fit(x, y.squeeze())

pred = clf.predict(x)

print("In-sample accuracy: {}".format(accuracy_score(y.squeeze(), pred)))

    /opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
      "avoid this warning.", FutureWarning)


    In-sample accuracy: 0.9397028122313643


Can you see the API pattern yet?

### Random Forest Models

Again, available in both [classification](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) and [regression](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor) flavors, these models are aggregations of many randomized Decision Trees.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(n_estimators=50)
clf = clf.fit(x, y.squeeze())

pred = clf.predict(x)

print("In-sample accuracy: {}".format(accuracy_score(y.squeeze(), pred)))

    In-sample accuracy: 0.9748250030701215


There MUST be a pattern here...

Of course there is! We import our classifier (or regressor), then create an instance of that object. We can name it `clf` or anything else that we prefer. From there, the process is the same:
- Use the `.fit()` method, passing in the relevant data for our context
- Create predictions using our fitted model with `.predict()` and new exogenous data (or the old data to test in-sample fit)
- Measure the performance of our model with `accuracy_score`, or any other metric that can describe performance given a specific use case

### More from `sklearn`

Many other tools are also available to aid in the data cleaning process through `sklearn`. Some of these are:

- [Principal Component Analysis (PCA)](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA)
- [Factor Analysis](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FactorAnalysis.html#sklearn.decomposition.FactorAnalysis)
- Many [Cross-Validation Algorithms](http://scikit-learn.org/stable/modules/cross_validation.html)
- [Hyperparameter Tuning](http://scikit-learn.org/stable/modules/grid_search.html)
   - Finding the correct parameters for a decision tree or random forest, for example
- [Model Evaluation Tools](http://scikit-learn.org/stable/modules/model_evaluation.html)
- [Plotting decision trees](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html#sklearn.tree.plot_tree)


## Solve-it!

Using the wage data provided here (https://github.com/dustywhite7/pythonMikkeli/raw/master/exampleData/wagePanelData.csv), create a linear regression model to explain and/or predict wages. Your data set should be labeled `data` and your fitted model should be stored as `reg`. If you do not name the model correctly, you won't get any points!

Please put all your code for this exercise in the cell labeled `#si-linear-regression` file found below.

In [ ]:
#si-linear-regression

#Passed
import statsmodels.formula.api as smf
import pandas as pd, numpy as np

data = pd.read_csv("https://github.com/dustywhite7/pythonMikkeli/raw/master/exampleData/wagePanelData.csv")

data.columns

reg = smf.ols("log_wage ~ education + is_black + years_experience", data=data).fit()
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:               log_wage   R-squared:                       0.269
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     509.4
Date:                Thu, 05 Dec 2024   Prob (F-statistic):          6.34e-282
Time:                        00:21:13   Log-Likelihood:                -2037.4
No. Observations:                4165   AIC:                             4083.
Df Residuals:                    4161   BIC:                             4108.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            5.4900      0.034  

## Solve-it!

Import the pass/fail data for students in Portugal found here(https://github.com/dustywhite7/pythonMikkeli/raw/master/exampleData/passFailTrain.csv), and create a logistic regression model
using `statsmodels` that can estimate the likelihood of students passing or failing class. The dependent variable is contained in the column called `G3`, which takes the value `1` when the student has a passing final grade, and `0` otherwise.

Call your fitted model `reg`, and place all code for this exercise in the cell labeled `#si-logistic-regression` file found below.

In [ ]:
#si-logistic-regression

#for binary dependent variables use logit!

import statsmodels.formula.api as smf
import pandas as pd, numpy as np

data2 = pd.read_csv("https://github.com/dustywhite7/pythonMikkeli/raw/master/exampleData/passFailTrain.csv")

data2.columns

reg = smf.logit("G3 ~ studytime + sex + internet", data=data2).fit()
print(reg.summary())

Optimization terminated successfully.
         Current function value: 0.616319
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                     G3   No. Observations:                  296
Model:                          Logit   Df Residuals:                      292
Method:                           MLE   Df Model:                            3
Date:                Thu, 05 Dec 2024   Pseudo R-squ.:                 0.01393
Time:                        00:35:20   Log-Likelihood:                -182.43
converged:                       True   LL-Null:                       -185.01
Covariance Type:            nonrobust   LLR p-value:                    0.1610
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2610      0.485     -0.538      0.590      -1.211       0.689
studytime      0.3131      0.

## Solve-it!

Use the data on NFL franchise values included in the NFL Valuation data source (https://raw.githubusercontent.com/dustywhite7/Econ8320/master/AssignmentData/assignment12Data.csv) file to implement a Random Forest Classifier in sklearn using 100 trees to predict team-years when `Playoffs` takes the value `1` (when a team made the playoffs in that season).

- Use Patsy to create `x2` and `y2` matrices
- Create the classifier
- Fit the classifier, and store the fitted model with the name `playoffForest`

Place all code for this exercise in the cell labeled `#si-random-forest` file found below.

In [115]:
#si-random-forest
import pandas as pd
import numpy as np
import patsy as pt
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


data = pd.read_csv("https://raw.githubusercontent.com/dustywhite7/Econ8320/master/AssignmentData/assignment12Data.csv")

data

#do i need to make the samples?
data4 = data.sample(100)
data5 = data.sample(100)

y, x = pt.dmatrices("Playoffs ~ -1 + SuperBowl + Year ", data=data4)

model = sm.OLS(y,x).fit()
print(model.summary())

x2 = pt.build_design_matrices([x.design_info], data5)

y2 = pt.build_design_matrices([y.design_info], data5)






x1, x2, y1, y2 = train_test_split(x, y)

playoffForest_noFit = RandomForestClassifier(n_estimators = 100) #do have to provide one argument for the number of "trees" in our "fores"
playoffForest = playoffForest_noFit.fit(x1, y1)

pred = playoffForest.predict(x2)

print(accuracy_score(y2, pred))

                                 OLS Regression Results                                
Dep. Variable:               Playoffs   R-squared (uncentered):                   0.451
Model:                            OLS   Adj. R-squared (uncentered):              0.440
Method:                 Least Squares   F-statistic:                              40.27
Date:                Thu, 05 Dec 2024   Prob (F-statistic):                    1.72e-13
Time:                        01:59:43   Log-Likelihood:                         -64.823
No. Observations:                 100   AIC:                                      133.6
Df Residuals:                      98   BIC:                                      138.9
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [101]:
#testing code
#y1
import re

(bool(re.search(r'RandomForestClassifier', str(type(playoffForest)))), "You didn't make a random forest!")

(True, "You didn't make a random forest!")

--

--



# LECTURE NOTES





In [ ]:
#modeling through Statsmodels/Sklearn

#OLS

import statsmodels.api as sm  #this is the non formula methode for importing stats models that requires numbpy arrays
#this is easier because it allows for ineroperability when switching through regressions and machine learning


#startign with this option first
import statsmodels.formula.api as smf #easiest way to import stats models and allows us to use regression equatons (like r) in order to run regressions very easily

#for data sets
#1. No spaces in names, no symbols names, made up of letters and numbers in names (no numbers at the start)

In [ ]:
#preparing a dataset

import statsmodels.formula.api as smf
import pandas as pd, numpy as np

data = pd.read_csv("https://github.com/dustywhite7/Econ8320/blob/master/AssignmentData/assignment8Data.csv?raw=true")

In [ ]:
#data.columns

#model hourly wage a dependent variable ~ then independent variables (JUST LIKE R, yayyyy) [age and educations], then call on the dataset were pulling regression info
#from data = data
model = smf.ols("hrwage ~ age + educ", data=data)

In [ ]:
#doe the model fit (.fit) to find out parameters (beta coeficients)
modelFit = model.fit()

modelFit.summary() #this gives us our stats summary table

#from table we can see that as age increases we have a small increase in hourly wage
#as education level increases we have a non trivial increase in wages ($2.65 in wage for every education level increase)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 hrwage   R-squared:                       0.075
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     406.1
Date:                Wed, 04 Dec 2024   Prob (F-statistic):          2.65e-170
Time:                        22:01:34   Log-Likelihood:                -44037.
No. Observations:               10008   AIC:                         8.808e+04
Df Residuals:                   10005   BIC:                         8.810e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.7081      0.980     -7.867      0.000      -9.629      -5.787
age            0.1168      0.013      8.963      0.000       0.091       0.142
educ           2.6515      0.097     27.324      0.000       2.461       2.842
==============================================================================
Omnibus:                    19307.010   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         97351740.228
Skew:                          14.707   Prob(JB):                         0.00
Kurtosis:                     485.278   Cond. No.                         240.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
#now place the betas we found into the regression and solve for if age was 35 and education level was 11
-7.7+ 0.11*35 + 2.65*11

#expected hourly wage with those inputs based on this model is $25.299

25.299999999999997

In [ ]:
#now lets increase the compexcity of the equation
#data.columns #look at what other varibales are avaialble to us

#want to include age and age^2 (because we know that age is generally non-linear as it rises and then dips as you get older)
model = smf.ols("hrwage ~ age + I(age**2) + educ", data=data) #I(age**2)  transforms age into a square
modelFit = model.fit()

modelFit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 hrwage   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     316.3
Date:                Wed, 04 Dec 2024   Prob (F-statistic):          3.15e-196
Time:                        22:10:24   Log-Likelihood:                -43974.
No. Observations:               10008   AIC:                         8.796e+04
Df Residuals:                   10004   BIC:                         8.799e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     -23.1554      1.683    -13.756      0.000     -26.455     -19.856
age             0.9383      0.074     12.652      0.000       0.793       1.084
I(age ** 2)    -0.0090      0.001    -11.250      0.000      -0.011      -0.007
educ            2.4954      0.097     25.612      0.000       2.304       2.686
==============================================================================
Omnibus:                    19537.539   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        104604362.363
Skew:                          15.100   Prob(JB):                         0.00
Kurtosis:                     502.938   Cond. No.                     2.30e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.3e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#now place the betas we found into the regression and solve for if age was 35 and education level was 11
-7.7 + 0.9383*35 + -.009*35**2 + 2.65 *11

#now expected wage is 43.2655 with more complex model and teacher says this is closer to his actual wage so yay

#increase the information in our regression gives us a better prediction

43.2655

In [ ]:
#can also do log
model = smf.ols("np.log(I(hrwage+1)) ~ age + I(age**2) + educ", data=data) #I(age**2)  transforms age into a square, np.log is showing the model if we took log of hourlywage
modelFit = model.fit()

modelFit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(I(hrwage + 1))   R-squared:                       0.109
Model:                               OLS   Adj. R-squared:                  0.109
Method:                    Least Squares   F-statistic:                     409.7
Date:                   Wed, 04 Dec 2024   Prob (F-statistic):          4.95e-251
Time:                           22:15:14   Log-Likelihood:                -13393.
No. Observations:                  10008   AIC:                         2.679e+04
Df Residuals:                      10004   BIC:                         2.682e+04
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.2335      0.079      2.946      0.003       0.078       0.389
age             0.0689      0.003     19.718      0.000       0.062       0.076
I(age ** 2)    -0.0007   3.78e-05    -19.704      0.000      -0.001      -0.001
educ            0.1188      0.005     25.882      0.000       0.110       0.128
==============================================================================
Omnibus:                     2271.958   Durbin-Watson:                   1.950
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5075.249
Skew:                          -1.297   Prob(JB):                         0.00
Kurtosis:                       5.333   Cond. No.                     2.30e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.3e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#The C() command indicates that we would like to consider the race variable as a Categorical variable, not a numeric variable.
model = smf.ols("np.log(I(hrwage+1)) ~ age + I(age**2) + C(educ)", data=data) #create categorical varibales for every education level, creates nonlinear relationship between education categories
modelFit = model.fit()

modelFit.summary()

#in theoriy since adding the c increase out r-squared a bit this should mean that this mafe rthe model better or sumthin

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(I(hrwage + 1))   R-squared:                       0.118
Model:                               OLS   Adj. R-squared:                  0.117
Method:                    Least Squares   F-statistic:                     121.7
Date:                   Wed, 04 Dec 2024   Prob (F-statistic):          8.85e-263
Time:                           22:17:49   Log-Likelihood:                -13344.
No. Observations:                  10008   AIC:                         2.671e+04
Df Residuals:                       9996   BIC:                         2.680e+04
Df Model:                             11                                         
Covariance Type:               nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.9770      0.199      4.900      0.000       0.586       1.368
C(educ)[T.2]     -0.0713      0.204     -0.350      0.726      -0.470       0.328
C(educ)[T.3]      0.0318      0.216      0.147      0.883      -0.391       0.455
C(educ)[T.4]     -0.1579      0.211     -0.750      0.453      -0.571       0.255
C(educ)[T.5]     -0.1769      0.198     -0.892      0.373      -0.566       0.212
C(educ)[T.6]      0.0011      0.184      0.006      0.995      -0.360       0.363
C(educ)[T.7]      0.0758      0.185      0.409      0.682      -0.287       0.439
C(educ)[T.8]      0.1268      0.186      0.683      0.494      -0.237       0.490
C(educ)[T.10]     0.4622      0.185      2.501      0.012       0.100       0.824
C(educ)[T.11]     0.7630      0.186      4.099      0.000       0.398       1.128
age               0.0678      0.004     19.289      0.000       0.061       0.075
I(age ** 2)      -0.0007    3.8e-05    -19.427      0.000      -0.001      -0.001
==============================================================================
Omnibus:                     2269.664   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5050.066
Skew:                          -1.298   Prob(JB):                         0.00
Kurtosis:                       5.319   Cond. No.                     1.70e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.7e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#robust standard errors

modelFit.get_robustcov_results(cov_type ='HC0').summary()

#now standard errors are more robust to protect against heteroskedasticity

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(I(hrwage + 1))   R-squared:                       0.118
Model:                               OLS   Adj. R-squared:                  0.117
Method:                    Least Squares   F-statistic:                     133.0
Date:                   Wed, 04 Dec 2024   Prob (F-statistic):          3.29e-286
Time:                           22:20:33   Log-Likelihood:                -13344.
No. Observations:                  10008   AIC:                         2.671e+04
Df Residuals:                       9996   BIC:                         2.680e+04
Df Model:                             11                                         
Covariance Type:                     HC0                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.9770      0.152      6.427      0.000       0.679       1.275
C(educ)[T.2]     -0.0713      0.156     -0.457      0.648      -0.377       0.235
C(educ)[T.3]      0.0318      0.155      0.205      0.838      -0.273       0.336
C(educ)[T.4]     -0.1579      0.164     -0.966      0.334      -0.478       0.163
C(educ)[T.5]     -0.1769      0.146     -1.210      0.226      -0.464       0.110
C(educ)[T.6]      0.0011      0.134      0.008      0.993      -0.261       0.263
C(educ)[T.7]      0.0758      0.135      0.563      0.573      -0.188       0.340
C(educ)[T.8]      0.1268      0.135      0.936      0.349      -0.139       0.392
C(educ)[T.10]     0.4622      0.134      3.450      0.001       0.200       0.725
C(educ)[T.11]     0.7630      0.136      5.622      0.000       0.497       1.029
age               0.0678      0.004     18.602      0.000       0.061       0.075
I(age ** 2)      -0.0007   4.19e-05    -17.607      0.000      -0.001      -0.001
==============================================================================
Omnibus:                     2269.664   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5050.066
Skew:                          -1.298   Prob(JB):                         0.00
Kurtosis:                       5.319   Cond. No.                     1.70e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.7e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

OTHER STATSMODEL MODELING OPTIONS

In [ ]:
#Modeling descrete outcomes

#Logit and Probit

#lets try a logistic regression instead of ols this time

#try to predict marrage based on number of children under 5
model = smf.logit("married ~ nchlt5", data=data)
modelFit = model.fit()

modelFit.summary()

#tells that # of children has a strong positive relationship with whether someone is married (so having children under 5 is a good indicator for predicting is someone is married)

Optimization terminated successfully.
         Current function value: 0.634598
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                married   No. Observations:                13712
Model:                          Logit   Df Residuals:                    13710
Method:                           MLE   Df Model:                            1
Date:                Wed, 04 Dec 2024   Pseudo R-squ.:                 0.02816
Time:                        22:26:00   Log-Likelihood:                -8701.6
converged:                       True   LL-Null:                       -8953.7
Covariance Type:            nonrobust   LLR p-value:                1.147e-111
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4602      0.018     24.906      0.000       0.424       0.496
nchlt5         1.2554      0.072     17.552      0.000       1.115       1.396
==============================================================================
"""

In [ ]:
#add to see how different educational level (made categorical) are associated with marital status
model = smf.logit("married ~ nchlt5 + C(educ)", data=data)
modelFit = model.fit()

modelFit.summary()
#we can see that everything after T.8 (which is highschool graduuate) has positive coefficient
#this would mean that being a highschool grad or more steadily increases the likelyhood of somebody being married

Optimization terminated successfully.
         Current function value: 0.624782
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                married   No. Observations:                13712
Model:                          Logit   Df Residuals:                    13701
Method:                           MLE   Df Model:                           10
Date:                Wed, 04 Dec 2024   Pseudo R-squ.:                 0.04319
Time:                        22:28:16   Log-Likelihood:                -8567.0
converged:                       True   LL-Null:                       -8953.7
Covariance Type:            nonrobust   LLR p-value:                1.073e-159
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.4283      0.330      1.298      0.194      -0.219       1.075
C(educ)[T.2]     -0.1499      0.355     -0.422      0.673      -0.846       0.547
C(educ)[T.3]      0.0487      0.386      0.126      0.899      -0.707       0.804
C(educ)[T.4]     -0.3370      0.366     -0.921      0.357      -1.054       0.380
C(educ)[T.5]     -0.9590      0.353     -2.716      0.007      -1.651      -0.267
C(educ)[T.6]     -0.0678      0.331     -0.205      0.838      -0.717       0.581
C(educ)[T.7]     -0.1913      0.333     -0.574      0.566      -0.844       0.461
C(educ)[T.8]      0.2012      0.335      0.601      0.548      -0.455       0.858
C(educ)[T.10]     0.2421      0.333      0.727      0.467      -0.410       0.895
C(educ)[T.11]     0.8236      0.338      2.434      0.015       0.160       1.487
nchlt5            1.1981      0.072     16.689      0.000       1.057       1.339
=================================================================================
"""

In [ ]:
#we can also use count models (we dont just have to use binary models)

#now trying to see how well hourly wage can predict the number of children under 5
model = smf.poisson("nchlt5~ hrwage", data=data[data['nchlt5']>0]) #the last part is making so we are only looking at the data where number of children under 5 is greaeter than
#zero idk why but for some reason poisson cannot take this variable unless it is non-zero

modelFit = model.fit()

modelFit.summary()

#hourlywafge increase have a small positive effect on the number of children (.0005), so its has a little affect but not statistically significant

Optimization terminated successfully.
         Current function value: 1.198935
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:                 nchlt5   No. Observations:                 1259
Model:                        Poisson   Df Residuals:                     1257
Method:                           MLE   Df Model:                            1
Date:                Wed, 04 Dec 2024   Pseudo R-squ.:               3.527e-05
Time:                        22:39:33   Log-Likelihood:                -1509.5
converged:                       True   LL-Null:                       -1509.5
Covariance Type:            nonrobust   LLR p-value:                    0.7442
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2807      0.037      7.638      0.000       0.209       0.353
hrwage         0.0005      0.001      0.329      0.742      -0.002       0.003
==============================================================================
"""

USING THE PATSY LIBRARY: Using regression equations

In [ ]:
#lets us the use for stats models and machine learning for building out our matrixes

#lets us create a pipeilne where we can put future data in identical structure (basically it will adjust itself around any changes in new)



import statsmodels.formula.api as smf
import patsy as pt
import pandas as pd
import numpy as np

data = pd.read_csv("https://github.com/dustywhite7/Econ8320/blob/master/AssignmentData/assignment8Data.csv?raw=true")

# To create y AND x matrices
y, x = pt.dmatrices("hhincome ~ married + nchlt5 + educ", data = data)



In [ ]:
y

DesignMatrix with shape (13712, 1)
  hhincome
     44000
     44000
     15000
     72000
     72000
     73560
     73560
    119500
    119500
    146200
    146200
     60000
    159000
    159000
     16800
     71000
     71000
     57400
     57400
    119700
    119700
     80200
     80200
     24600
     52450
     52450
     52450
     66000
     66000
     49900
  [13682 rows omitted]
  Terms:
    'hhincome' (column 0)
  (to view full data, use np.asarray(this_obj))

In [ ]:
x #same number of rows as y but 4 columsn even though we only passed through 3 variables. This because the first column is the intercept

DesignMatrix with shape (13712, 4)
  Intercept  married  nchlt5  educ
          1        1       0     7
          1        1       0     6
          1        0       0     4
          1        1       0     6
          1        1       0     8
          1        1       0    11
          1        1       0    10
          1        1       0     7
          1        1       0     7
          1        1       0     7
          1        1       0     6
          1        0       0     7
          1        1       0     8
          1        1       0    11
          1        0       0     6
          1        1       0    10
          1        1       0    11
          1        1       0     6
          1        1       0     6
          1        1       0     7
          1        1       0    10
          1        1       0     6
          1        1       0     6
          1        0       0     4
          1        1       1     6
          1        1       1     6
          1        0

In [ ]:
# To create ONLY an x matrix
#x = pt.dmatrix("~ year + educ + married + age",
		#data = data)

In [ ]:
# we can now use these to d matrix algebra

x.T @ x #x transposed at x

array([[ 13712.,   8786.,   1916., 101108.],
       [  8786.,   8786.,   1727.,  66477.],
       [  1916.,   1727.,   3046.,  15496.],
       [101108.,  66477.,  15496., 804014.]])

In [ ]:
np.linalg.inv(x.T @ x) @ x.T @ y #this will give us back the beta coefs in our ols model


array([[-6294.29708105],
       [27697.520517  ],
       [-7336.06481466],
       [ 8925.63788719]])

In [ ]:

import statsmodels.api as sm

model = sm.OLS(y,x)
modelFit = model.fit()

modelFit.summary() #now we can run the regression using the y and x and now we can use them in any other model (not just OLS) thanks to patsy

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               hhincome   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     711.3
Date:                Wed, 04 Dec 2024   Prob (F-statistic):               0.00
Time:                        23:18:40   Log-Likelihood:            -1.7032e+05
No. Observations:               13712   AIC:                         3.406e+05
Df Residuals:                   13708   BIC:                         3.407e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -6294.2971   1958.258     -3.214      0.001   -1.01e+04   -2455.843
married      2.77e+04   1090.050     25.409      0.000    2.56e+04    2.98e+04
nchlt5     -7336.0648   1159.508     -6.327      0.000   -9608.859   -5063.271
educ        8925.6379    251.076     35.549      0.000    8433.494    9417.782
==============================================================================
Omnibus:                    10373.944   Durbin-Watson:                   1.058
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           340766.392
Skew:                           3.325   Prob(JB):                         0.00
Kurtosis:                      26.500   Cond. No.                         30.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

NOW we are going to pass new data and build the same design matrix SUPER IMPORTANT

In [ ]:
#made 2 new sample datas from our original data
data1 = data.sample(1000)
data2 = data.sample(1000)

#now we'll maek regression run on the first datat set
y, x = pt.dmatrices("hhincome ~ married + nchlt5 + educ", data = data1)


import statsmodels.api as sm

model = sm.OLS(y,x).fit()
#modelFit = model.fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               hhincome   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     41.45
Date:                Wed, 04 Dec 2024   Prob (F-statistic):           3.03e-25
Time:                        23:43:30   Log-Likelihood:                -12393.
No. Observations:                1000   AIC:                         2.479e+04
Df Residuals:                     996   BIC:                         2.481e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1991.4203   7049.300      0.282      0.778   -1.18e+04    1.58e+04
married     2.275e+04   3971.744      5.729      0.000     1.5e+04    3.05e+04
nchlt5     -6229.1414   4232.843     -1.472      0.141   -1.45e+04    2077.172
educ        8069.8972    898.051      8.986      0.000    6307.607    9832.187
==============================================================================
Omnibus:                      613.565   Durbin-Watson:                   1.911
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6986.455
Skew:                           2.656   Prob(JB):                         0.00
Kurtosis:                      14.809   Cond. No.                         30.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
#now lets say that data 1 was our original data and we want to make predicitions on data 2

x2 = pt.build_design_matrices([x.design_info], data2) #build our nex x based on the old one

In [ ]:
x

DesignMatrix with shape (1000, 4)
  Intercept  married  nchlt5  educ
          1        0       0    10
          1        0       0     6
          1        0       0    10
          1        1       0     8
          1        1       0     6
          1        1       0     6
          1        0       0     6
          1        1       0     8
          1        1       0     6
          1        0       0     6
          1        1       0    10
          1        1       1    11
          1        1       0     6
          1        0       0    10
          1        0       0     6
          1        1       0     3
          1        0       0     6
          1        1       1     8
          1        1       0     7
          1        0       0     6
          1        0       0     6
          1        0       0    10
          1        0       0    10
          1        1       0     6
          1        0       1    10
          1        0       0    10
          1        0 

In [ ]:
x2
 #now x2 is also 1000 rowsa and 4 colums like x and we didnt need a regression equation to build as it just took the structure and process as x
#can make predictions even as new data rolls in


#model.predict(x2)

[DesignMatrix with shape (1000, 4)
   Intercept  married  nchlt5  educ
           1        1       2     7
           1        0       0     6
           1        1       0     6
           1        1       0     6
           1        1       0     6
           1        1       0     7
           1        1       0    10
           1        1       0     6
           1        0       0     6
           1        1       0     7
           1        1       0    10
           1        1       1     8
           1        1       0     6
           1        1       0    10
           1        0       0    11
           1        0       0     6
           1        0       0     4
           1        1       0    10
           1        1       0     6
           1        1       0     6
           1        0       0    10
           1        0       0    10
           1        1       0     6
           1        1       0    11
           1        0       0     7
           1        0       0

In [ ]:
model.predict(x2) #this gives us predications for x2 even though we didnt train it on x2, however now that it is the same shape as x1, you get
#reasonable predictions on x2 becaue it structed the same as x

#this is why patsy is good, yay

array([[ 68777.2533239 ,  50410.80381352,  73165.63891515,
         73165.63891515,  73165.63891515,  81235.53616496,
        105445.22791438,  73165.63891515,  50410.80381352,
         81235.53616496, 105445.22791438,  83076.29199424,
         73165.63891515, 105445.22791438,  90760.29006256,
         50410.80381352,  34271.00931391, 105445.22791438,
         73165.63891515,  73165.63891515,  82690.39281275,
         82690.39281275,  73165.63891515, 113515.12516419,
         58480.70106333,  58480.70106333,  73165.63891515,
         56318.97048284,  34656.9084954 ,  50410.80381352,
         58480.70106333,  73165.63891515, 113515.12516419,
        105445.22791438,  73165.63891515, 113515.12516419,
         73165.63891515,  89305.43341477,  32816.15266612,
         73165.63891515,  50410.80381352, 105445.22791438,
         18131.21481429,  89305.43341477,  73165.63891515,
         90760.29006256,  92986.94507332,  82690.39281275,
         48955.94716573, 105445.22791438,  42340.9065637

MACHINE LEARNING

In [ ]:
#scikit-learn in the library most used in pythin for machine learn

#predictive modeling

import pandas as pd
import numpy as np
import patsy as pt
import statsmodels.api as sm

# Import some data...
data = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/master/DataSets/occupancyTrain.csv")


# Build x, y matrices
y, x = pt.dmatrices("Occupancy ~ -1 + Light + CO2", data=data) #the -1 in patsy means no intercept column, most machine learning models dont like intercept terms



In [ ]:
y # represents 1 and 0 to tells us when the room is or isnt occupie

DesignMatrix with shape (8143, 1)
  Occupancy
          1
          1
          1
          1
          1
          1
          1
          1
          1
          1
          1
          1
          1
          1
          1
          1
          0
          0
          0
          0
          0
          0
          0
          0
          0
          0
          0
          0
          0
          0
  [8113 rows omitted]
  Terms:
    'Occupancy' (column 0)
  (to view full data, use np.asarray(this_obj))

In [ ]:
x #notice not intercept column on the 2 variables, tryign to use these to predict room occupancy

DesignMatrix with shape (8143, 2)
  Light        CO2
  426.0  721.25000
  429.5  714.00000
  426.0  713.50000
  426.0  708.25000
  426.0  704.50000
  419.0  701.00000
  419.0  701.66667
  419.0  699.00000
  419.0  689.33333
  419.0  688.00000
  419.0  690.25000
  419.0  691.00000
  419.0  683.50000
  419.0  687.50000
  419.0  686.00000
  418.5  680.50000
    0.0  681.50000
    0.0  685.00000
    0.0  685.00000
    0.0  689.00000
    0.0  689.50000
    0.0  689.00000
    0.0  691.00000
    0.0  688.00000
    0.0  689.50000
    0.0  689.00000
    0.0  685.66667
    0.0  687.00000
    0.0  688.00000
    0.0  670.00000
  [8113 rows omitted]
  Terms:
    'Light' (column 0)
    'CO2' (column 1)
  (to view full data, use np.asarray(this_obj))

In [ ]:
#will be doing classificiation trees

from sklearn import tree #imports all tree based code
from sklearn.metrics import accuracy_score #accuracy score is what well use to test performance of predictive model (% of observations correctly claisifed)
from sklearn.model_selection import train_test_split

x1, x2, y1, y2 = train_test_split(x,y) #split x and y varables, shuffle them, and randomly assign observations to a training and testing data set

#we have a binary classification problem (either the room is or isnt claissified ) so accuracy score is going to be the % of observations we corectly classify if the
#room is occupioed

clf = tree.DecisionTreeClassifier() #this creates classifier by initiating an instance of the classifier object, now clf is classifier object

#now need to modify the decision tree classifier object. When using sklearn we put x and y into the fit function and x comes before y
clf = clf.fit(x1, y1) #fit the classifier

#make predictions ased on the data we have
pred = clf.predict(x2) #predict based on x2

print(accuracy_score(y2, pred)) #measure accuray score based on y2v <-here we are comparing the predictions in pred with x2 to the true variables which come from y2
#this should give us an acuracy score

#the score means that based on co2 and light we are going to be about 98% accurate in determing is a room is occupied  (tells us how well our mode performed)

0.987721021611002


In [ ]:
x1

array([[   0.        , 1392.33333333],
       [   0.        ,  441.5       ],
       [ 405.        ,  655.5       ],
       ...,
       [ 454.        , 1338.        ],
       [   0.        ,  432.5       ],
       [   0.        ,  433.5       ]])

In [ ]:
#now we can do a support vector machine

from sklearn import svm
from sklearn.metrics import accuracy_score

clf = svm.SVC()
clf = clf.fit(x1, y1)

pred = clf.predict(x2)

print(accuracy_score(y2, pred))

#This one gave us a highrt accuracy score

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9891944990176817


In [ ]:
#now try a random forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(n_estimators = 100) #do have to provide one argument for the number of "trees" in our "fores"
clf = clf.fit(x1, y1)

pred = clf.predict(x2)

print(accuracy_score(y2, pred))

#this model does a little better than the other ones

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.9896856581532416


In [ ]:
#now we can go back and look at our predictions to see which observations are successes and failures and go from there
pred

array([0., 1., 0., ..., 0., 0., 0.])